In [8]:
import igraph
import networkx as nx 
import numpy as np
import pandas as pd 
import copy

#### Use the way I process it 

In [9]:
def preprocess(df):
    keep_var = ['countrycode','counterpart_code','country','counterpart','year','CDIS_IAD','CPIS_IAP','CPIS_IAPD','CPIS_IAPE','loans_dep','total_claims']
    df = df[keep_var]                           ## keep only used variables 
    df = df.replace(np.nan,0)                   ## turn na to zero 
    num = df._get_numeric_data()
    num[num < 0] = 0                            ## turn negative to zero 
    df['total'] = df['total_claims']
    df.fillna(0,inplace=True)
    
    return df 

def load_graph_nx(df,year,var):
    ## clean the data first 
    df_y = df[df['year']==year].copy()
    df_y.fillna(0,inplace=True)
    df_y = df_y[df_y[var]>0]
    #G = nx.from_pandas_dataframe(df_y, source="country", target="counterpart", edge_attr=[var],create_using=nx.DiGraph())
    G = nx.from_pandas_dataframe(df_y, source="counterpart", target="country", edge_attr=[var],create_using=nx.DiGraph())
    return G


In [10]:
## read pre-processed data from stata
df = pd.read_stata('../data/0_CPIS_CDIS_BIS_USTIC_merged_fixed5.dta')
df = preprocess(df)

In [11]:
#df[(df.country == 'United States') & (df.counterpart == 'United Kingdom')&(df.year == 2015)]
df[(df.country == 'China, P.R.: Mainland') & (df.counterpart == 'United States')&(df.year == 2015)]

,countrycode,counterpart_code,country,counterpart,year,CDIS_IAD,CPIS_IAP,CPIS_IAPD,CPIS_IAPE,loans_dep,total_claims,total
88318,924.0,111.0,"China, P.R.: Mainland",United States,2015,25952.001953,1844020.0,1513531.0,330489.0,97638.0,1967610.0,1967610.0


#### Igraph

### Use the adj matrix

In [17]:
year = 2015
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt ('../data/adj/AM4_all_nodes_aggregate'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv('../data/adj/all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(aggregate_am))
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [18]:
i =  Aggregate_g.get_eid('United States','China  P.R.: Mainland')
Aggregate_g.es[i]

igraph.Edge(<igraph.Graph object at 0x7f2c662d3228>, 13554, {'weight': 1967610.001953125})

- I change the norm = False, now we are getting the same edge weight 

In [20]:
PR = Aggregate_g.personalized_pagerank(weights=Aggregate_g.es["weight"])
pr= zip(Aggregate_g.vs["name"],PR)
c_df = pd.DataFrame(list(pr),columns=['country','centrality']) 
c_df.sort_values(by='centrality',ascending=0,inplace=True)                 ## sort it 
c_df.head(20)

,country,centrality
201,United States,0.138609
200,United Kingdom,0.081233
116,Luxembourg,0.070937
137,Netherlands,0.061216
68,France,0.053207
74,Germany,0.051236
98,Japan,0.035362
41,China P.R.: Hong Kong,0.028973
181,Switzerland,0.026556
43,China P.R.: Mainland,0.022566


In [21]:
c_df[c_df.country=="United Kingdom"]

,country,centrality
200,United Kingdom,0.081233


- and the results are different